In [1]:
import os
import polars as pl
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from tensorflow.keras import layers, models, optimizers, regularizers, callbacks



In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
def get_generator_v3(dataframe, weights, feature_names, label_name, shuffle=True, batch_size=8192):
    def generator():
        indices = np.arange(len(dataframe))
        if shuffle:
            np.random.shuffle(indices)

        num_batches = len(indices) // batch_size + (1 if len(indices) % batch_size > 0 else 0)

        for i in range(num_batches):
            start_index = i * batch_size
            end_index = min((i + 1) * batch_size, len(dataframe))
            current_indices = indices[start_index:end_index]

            features = dataframe.iloc[current_indices][feature_names].values
            labels = dataframe.iloc[current_indices][label_name].values
            if weights is not None:
                weights_batch = weights.iloc[current_indices].values
            else:
                weights_batch = np.ones(len(labels), dtype=np.float32)

            yield features, labels.reshape(-1, 1), weights_batch

    return generator


In [4]:

def prepare_dataset(dataframe, weights, feature_names, label_name, batch_size=8192, shuffle=True):
    num_features = len(feature_names)

    output_signature = (
        tf.TensorSpec(shape=(None, num_features), dtype=tf.float32),
        tf.TensorSpec(shape=(None, 1), dtype=tf.float32),
        tf.TensorSpec(shape=(None,), dtype=tf.float32)
    )

    dataset = tf.data.Dataset.from_generator(
        get_generator_v3(dataframe, weights, feature_names, label_name, shuffle, batch_size),
        output_signature=output_signature
    )

    return dataset

In [5]:
def create_model(input_dim, lr, weight_decay):
    # Create a sequential model
    model = models.Sequential()

    # Start with the first hidden layer with BatchNorm1d, SiLU, Dropout, and Linear layers
    # Input layer


    model.add(layers.Input(shape=(input_dim,)))


    model.add(layers.BatchNormalization())
    model.add(layers.Activation('swish'))
    model.add(layers.Dropout(0.1))  # Assuming dropouts[1] is valid
    model.add(layers.Dense(128, kernel_regularizer=regularizers.l2(weight_decay)))

    model.add(layers.BatchNormalization())
    model.add(layers.Activation('swish'))
    model.add(layers.Dropout(0.1))  # Assuming dropouts[1] is valid
    model.add(layers.Dense(64, kernel_regularizer=regularizers.l2(weight_decay)))

    # Output layer
    model.add(layers.Dense(1, activation='tanh'))

    # Compile model with Mean Squared Error loss
    # model.compile(optimizer=optimizers.Adam(learning_rate=lr), loss='mse', metrics=[WeightedR2()])
    model.compile(optimizer=optimizers.Adam(learning_rate=lr),
                  loss='mse',
                  metrics=[tf.keras.metrics.R2Score(class_aggregation='uniform_average')])
    return model


In [6]:

feature_names = [f"feature_{i:02d}" for i in range(79)] + [f"responder_{idx}_lag_1" for idx in range(9)]
label_name = 'responder_6'
weight_name = 'weight'


In [7]:
start_id = 1400
end_id = 1669
folder_paths = [
    f"/content/drive/MyDrive/JaneSt/CustomData_1/training/date_id={date_id}/*.parquet"
    for date_id in range(start_id, end_id + 1)
]
lazy_frames = [pl.scan_parquet(path) for path in folder_paths]
combined_lazy_df = pl.concat(lazy_frames)
df = combined_lazy_df.collect().to_pandas()




valid = pl.scan_parquet(f"/content/drive/MyDrive/JaneSt/CustomData_1/validation").collect().to_pandas()




In [8]:
# df = pd.concat([df, valid]).reset_index(drop=True)
df[feature_names] = df[feature_names].ffill().fillna(0)
valid[feature_names] = valid[feature_names].ffill().fillna(0)

In [9]:
# symbol_cluster1 = [10, 0, 16, 1, 7, 35, 9, 33]

# df = df[df['symbol_id'].isin(symbol_cluster1)]
# valid = valid[valid['symbol_id'].isin(symbol_cluster1)]


In [10]:
X_train = df[ feature_names ]
y_train = df[ label_name ]
w_train = df[ "weight" ]
X_valid = valid[ feature_names ]
y_valid = valid[ label_name ]
w_valid = valid[ "weight" ]

# train_dataset = prepare_dataset(df, w_train)
# valid_dataset = prepare_dataset(valid, w_valid)

train_dataset = prepare_dataset(df, w_train, feature_names, label_name, batch_size=8129)
valid_dataset = prepare_dataset(valid, w_valid, feature_names, label_name, batch_size=8129)




In [11]:

lr = 0.01
weight_decay = 5e-4

input_dim = df[feature_names].shape[1]
model = create_model(input_dim=input_dim, lr = lr, weight_decay=weight_decay)
model.summary()


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ batch_normalization                  │ (None, 88)                  │             352 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation (Activation)              │ (None, 88)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 88)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │          11,392 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_1                │ (None, 128)                 │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_1 (Activation)            │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 1)                   │              65 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 20,577 (80.38 KB)

 Trainable params: 20,145 (78.69 KB)

 Non-trainable params: 432 (1.69 KB)

In [14]:
ca = [
    tf.keras.callbacks.EarlyStopping(monitor='val_r2_score', patience=50, mode='max'),
    tf.keras.callbacks.ModelCheckpoint(
        filepath='/content/drive/MyDrive/JaneSt/TeamShare/models/model2/tf_nn_bas_{epoch:02d}.keras',
        monitor='val_loss',
        save_best_only=False,  # Set to False to save every epoch
        save_freq='epoch'  # Save at the end of each epoch
    ),
    tf.keras.callbacks.ReduceLROnPlateau(
        monitor='val_r2_score',
        factor=0.1,
        patience=20,
        verbose=1,
        min_lr=1e-6
    )
]

In [ ]:
model.fit(
    train_dataset.map(lambda x, y, w: (x, y, {'sample_weight': w})),
    epochs=100,
    validation_data=valid_dataset.map(lambda x, y, w: (x, y, {'sample_weight': w})),
    callbacks=ca
)


Epoch 1/100
   1220/Unknown 59s 48ms/step - loss: 1.6672 - r2_score: 8.1928e-04

/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


1220/1220 ━━━━━━━━━━━━━━━━━━━━ 67s 55ms/step - loss: 1.6672 - r2_score: 8.2304e-04 - val_loss: 1.0744 - val_r2_score: 0.0031 - learning_rate: 0.0100
Epoch 2/100


/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


1220/1220 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - loss: 1.6252 - r2_score: 0.0074

...

/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


1220/1220 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - loss: 1.5909 - r2_score: 0.0233

/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


1220/1220 ━━━━━━━━━━━━━━━━━━━━ 62s 51ms/step - loss: 1.5909 - r2_score: 0.0233 - val_loss: 1.0695 - val_r2_score: 0.0044 - learning_rate: 1.0000e-05


/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


In [ ]:

# Assume X_new is your new data you want to make predictions on
# This should be a NumPy array or a Tensor with shape (num_samples, num_features)
X_new = X_valid.to_numpy()  # Replace with actual data

# Make predictions
predictions = model.predict(X_new)
r2_metric = tf.keras.metrics.R2Score(class_aggregation='uniform_average')

if not isinstance(y_valid, np.ndarray):
    y_valid = y_valid.to_numpy()  # Convert to numpy array if it is a DataFrame

r2_metric.update_state(y_true=y_valid, y_pred=predictions)
r2_score_value = r2_metric.result().numpy()

print(f"R² Score on validation data: {r2_score_value}")


33820/33820 ━━━━━━━━━━━━━━━━━━━━ 39s 1ms/step
R² Score on validation data: 0.004446208477020264
